### Import libraries

In [1]:
import re
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse

import csv
import os
import pandas as pd

import time

/Users/phong-nh/Desktop/School/NLP/NLP/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/pb/6l_50wh16f19f6_cj43qm8_h0000gn/T/ipykernel_54843/359751531.py:13: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


#### Excute Selenium 
<blockquote>Paranm url : url to get selenium</blockquote> 
<blockquote>Return selenium object from url</blockquote> 

In [3]:
#url = "https://www.thegioididong.com/"
#Using selenium 

def excute_selenium(url):
    chrome_options = Options()
    chrome_options.add_argument("--incognito")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    time.sleep(2)
    return driver

In [4]:
def remove_last_slash(link):
    # Kiểm tra xem đường dẫn có kết thúc bằng dấu "/" không
    if link.endswith("/"):
        # Nếu có, loại bỏ dấu "/" cuối cùng bằng cách cắt chuỗi từ đầu đến ký tự thứ hai từ cuối
        remove_link = link[:-1]
        return remove_link
    return link

In [5]:
def remove_first_slash(link):
    if link.startswith("/"):
        remove_link = link[1:]
        return remove_link
    return link

print(remove_first_slash("/may-tinh-bang"))

may-tinh-bang


In [6]:
def extract_path_and_remove_query_params(url):
    parsed_url = urlparse(url)
    path_without_query = parsed_url.path.split('?')[0]
    path_parts = path_without_query.split('/')
    category = path_parts[1]
    product_name = path_parts[2]
    return category, product_name

#### Crawl category

In [7]:
def crawl_category(base_link, remove_key_word):
    driver = excute_selenium(base_link)
    content = driver.page_source
    result = []
    soup = BeautifulSoup(content, "html.parser")
    menu = soup.find('ul', class_='main-menu')
    list_a =  menu.findChildren('a')
    base_link_remove_last_slash = remove_last_slash(base_link)
    for i in list_a:
        if not any (word in i.get('href') for word in remove_key_word):
            cate_link = base_link_remove_last_slash + i.get('href')
            result.append([remove_first_slash(i.get('href')), cate_link])
            print([remove_first_slash(i.get('href')), cate_link])
            
    return result

In [8]:
def split_url(url):
    # Sử dụng biểu thức chính quy để tìm vị trí của kí tự số cuối cùng
    match = re.search(r'\d+$', url)
    if match:
        # Nếu tìm thấy, lấy vị trí của kí tự số cuối cùng
        poi = match.start()
        # Tách chuỗi thành hai phần
        prefix = url[:poi]
        endfix = url[poi:]
        return prefix, endfix
    else:
        # Nếu không tìm thấy kí tự số cuối cùng, trả về None cho cả hai phần
        return url, ""

#### Crawl max page of category

In [9]:
def get_end_page_from_category(url):
    #first find url
    driver = excute_selenium(url)
    category_url = ""
    try:
        driver.find_element(By.CLASS_NAME,'view-more')
        driver.find_element(By.CLASS_NAME,'view-more > a').click()
        time.sleep(2)
        category_url = driver.current_url
    except:
        category_url = url

    pre, end = split_url(category_url)
    if pre != "" and end != "":
        stop = False
        end_url = pre
        count = int(end)
        while stop == False:
            driver.get(end_url + str(count))     
            try:
                driver.find_element(By.CLASS_NAME, 'view-more')
                driver.find_element(By.CLASS_NAME,'view-more > a').click()
                time.sleep(2)
                count += 1
                end_url = driver.current_url
            except:
                stop = True
                
        end = count
        print(pre + str(end))
    return pre + str(end)

In [10]:
def crawl_list(base_link, url):
    driver =  excute_selenium(url)
    content = driver.page_source
    remove_key_words = ['javascript:void(0)','#']
    list_href = []
    soup = BeautifulSoup(content, "html.parser")  
    
    list_products = soup.find('ul', class_='listproduct')
    base_link_remove_last_slash = remove_last_slash(base_link)
    if list_products != None:
        li_products = list_products.findChildren('li')
        for li in li_products:
            a_products = li.findChildren('a')
            for a in a_products:
                if not any (word in a.get('href') for word in remove_key_words):
                    product_url = base_link_remove_last_slash + a.get('href')
                    list_href.append([a.get("data-name"), a.get('href'), product_url])
            
        print(f"Get {len(li_products)} from {url}")
        
    return list_href

#### Save Data

In [11]:
def write_to_csv(data, path, column_names, index = True, append=False):
    if data == None: 
        return "Data is null"
    if index == True:
        #Đánh index cho mỗi record
        data = [(i+1, item) for i, item in enumerate(data)]
    # check xem có phải là chế độ ghi liên tiếp hay không
    if append:
        # Tồn tại file đó hay không
        if not os.path.exists(path):
            # Nếu tập tin chưa tồn tại, tạo DataFrame từ dữ liệu và ghi vào tập tin
            df = pd.DataFrame(data, columns=column_names)
            df.to_csv(path, index=False)
            return "Data has written to csv file"
        df = pd.DataFrame(data)
        df.to_csv(path, mode='a', header=False, index=False)
        
    else:   
        if not os.path.exists(path):
            # Nếu tập tin chưa tồn tại, tạo DataFrame từ dữ liệu và ghi vào tập tin
            df = pd.DataFrame(data, columns=column_names)
            df.to_csv(path, index=False)
            return "Data has written to csv file"
        else:
            os.remove(path)
            print ("Found a csv file, deleting it...")
            df = pd.DataFrame(data, columns=column_names)
            df.to_csv(path, index=False)
            return "Data has written to csv file"

#### Read Data

In [12]:
def read_csv(path, is_read_header = True):
    data = []
    if not os.path.exists(path):
        print(f"File '{path}' does not exist.")
        return data
    # Mở tập tin CSV để đọc
    with open(path, 'r', newline='') as file:
        reader = csv.reader(file)
        if is_read_header: 
            next(reader)
        # Duyệt qua từng hàng trong tập tin CSV
        for row in reader:
            data.append(row)
    return data

#### Main
##### 1. Crawl all category of thegioididong and save to csv file
##### 2. Crawl all end page (find button view-more at the end of the page')
##### 3. Crawl all product link from end page link

In [13]:
base_link = "https://www.thegioididong.com/"

# removing_keyword = ['op-lung', 'mieng-dan', 'ldp', 'tien-ich/', 'gia-do-dien-thoai', 'tui-dung-airpods', 'tui-chong-soc', 'sim-so-dep']

# # Category crawl
# categories = crawl_category(base_link, removing_keyword)
# print(write_to_csv(categories, "categories.csv",["category", "full link"], False))

In [14]:
# end_link crawl
# end_page_link = []
# for i in  read_csv("categories.csv", True):
#     end_page_link.append([i[0], get_end_page_from_category(i[1])])
# print(write_to_csv(end_page_link, "end_page_link.csv", ["category", "full link"], False))

In [15]:
# product link crawl
# for i in read_csv("end_page_link.csv", True):
#     data = crawl_list(base_link, i[1])
#     write_to_csv(data, "product.csv", ["name", "category", "Link"], False, True)
    
# crawl_list(base_link, "https://www.thegioididong.com/phan-mem#c=85&o=14&pi=1")

In [16]:
pattern = '[^a-z0-9A-Z_ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂưăạảấầẩẫậắằẳẵặẹẻẽềềểỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễếệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳýỵỷỹ]+'
def preprocess_comment(comment):
    comment = comment.strip()
    comment = comment.replace('  ', ' ');
    return re.sub(pattern,' ', comment).lower()

In [17]:
def crawl_comment(base_url, category, product):
    data = []
    count = 0
    
    url = base_url + category + '/' + product 
    print(url)
    driver = excute_selenium(url)
    time.sleep(1)
    try:
        driver.find_element(By.CLASS_NAME, 'btn-view-all')
        url = base_url + category + '/' + product + '/danh-gia?page='
        number_page = 1
        while True:
            driver = excute_selenium(url + str(number_page))
            time.sleep(1)
            content = driver.page_source
            soup = BeautifulSoup(content, "html.parser")
            ul_comments = soup.find('ul', class_='comment-list')
            if ul_comments is None:
                print(f"Crawl {base_url + category + '/' + product } done!! Found {count} comments")
                break
            div_comments = ul_comments.find_all('div', {"class": "cmt-content"})
            for div_comment in div_comments:
                comment = preprocess_comment(div_comment.text)
                if comment != "":
                    data.append([category, product, comment])
                    count += 1
            number_page = number_page + 1
    except:
        content = driver.page_source
        soup = BeautifulSoup(content, "html.parser")
        ul_comments = soup.find('ul', class_='comment-list')
        if ul_comments is None:
            print(f"Crawl {base_url + category + '/' + product } done!! Found {count} comments")
        else:
            div_comments = ul_comments.find_all('p', {"class": "cmt-txt"})
            for div_comment in div_comments:
                comment = preprocess_comment(div_comment.text)
                if comment != "":
                    data.append([category, product, comment])
                    count += 1
    
    return data

In [18]:
products = read_csv('product.csv', True)

# for i in products:
#     category, product_name = extract_path_and_remove_query_params(i[1])
#     comments = crawl_comment(base_link, category, product_name)
#     write_to_csv(comments, "comments.csv", ["category", "product", 'comments'], False, True)
# comments = crawl_comment(base_link, "dtdd", "samsung-galaxy-s24-ultra-5g")
# write_to_csv(comments, "comments1.csv", ["category", "product", 'comments'], False, True)

In [22]:
# Nếu có lỗi trong quá trình crawl thì chạy hàm này
def crawl_product_continue(current_product):
    product_list = read_csv('product.csv', True)
    df = pd.read_csv('product.csv')
    product_category = df["category"].tolist()
    try:
        target_index = product_category.index(current_product)
        products_after_target = product_list[target_index:]
        for i in products_after_target:
            category_continue, product_name_continue = extract_path_and_remove_query_params(i[1])
            comments = crawl_comment(base_link, category_continue, product_name_continue)
            write_to_csv(comments, "comments.csv", ["category", "product", 'comments'], False, True)
        return products_after_target
    except ValueError:
        print("không tìm thấy sản phẩm")
        return []

crawl_product_continue("/phan-mem/eset-nod32-antivirus-cho-windows-1-pc")

https://www.thegioididong.com/phan-mem/eset-nod32-antivirus-cho-windows-1-pc
Crawl https://www.thegioididong.com/phan-mem/eset-nod32-antivirus-cho-windows-1-pc done!! Found 11 comments
https://www.thegioididong.com/phan-mem/kaspersky-plus-chinh-hang
Crawl https://www.thegioididong.com/phan-mem/kaspersky-plus-chinh-hang done!! Found 4 comments
https://www.thegioididong.com/phan-mem/vieon-vip-chinh-hang-1-thang
Crawl https://www.thegioididong.com/phan-mem/vieon-vip-chinh-hang-1-thang done!! Found 445 comments
https://www.thegioididong.com/phan-mem/vieon-vip-chinh-hang-3-thang
Crawl https://www.thegioididong.com/phan-mem/vieon-vip-chinh-hang-3-thang done!! Found 20 comments
https://www.thegioididong.com/phan-mem/vieon-vip-06-thang-chinh-hang
Crawl https://www.thegioididong.com/phan-mem/vieon-vip-06-thang-chinh-hang done!! Found 18 comments
https://www.thegioididong.com/phan-mem/vieon-hbo-go-06-thang-chinh-hang
Crawl https://www.thegioididong.com/phan-mem/vieon-hbo-go-06-thang-chinh-hang d

[['ESET NOD32 Antivirus chính hãng',
  '/phan-mem/eset-nod32-antivirus-cho-windows-1-pc',
  'https://www.thegioididong.com/phan-mem/eset-nod32-antivirus-cho-windows-1-pc'],
 ['Kaspersky Plus 01 PC chính hãng',
  '/phan-mem/kaspersky-plus-chinh-hang',
  'https://www.thegioididong.com/phan-mem/kaspersky-plus-chinh-hang'],
 ['VieON VIP 01 tháng chính hãng',
  '/phan-mem/vieon-vip-chinh-hang-1-thang',
  'https://www.thegioididong.com/phan-mem/vieon-vip-chinh-hang-1-thang'],
 ['VieON VIP 03 tháng chính hãng',
  '/phan-mem/vieon-vip-chinh-hang-3-thang',
  'https://www.thegioididong.com/phan-mem/vieon-vip-chinh-hang-3-thang'],
 ['VieON VIP 06 tháng chính hãng',
  '/phan-mem/vieon-vip-06-thang-chinh-hang',
  'https://www.thegioididong.com/phan-mem/vieon-vip-06-thang-chinh-hang'],
 ['VieON HBO Go 06 tháng chính hãng',
  '/phan-mem/vieon-hbo-go-06-thang-chinh-hang',
  'https://www.thegioididong.com/phan-mem/vieon-hbo-go-06-thang-chinh-hang'],
 ['VieON VIP HBO Go_03 tháng chính hãng',
  '/phan-me